In [1]:
# -*- coding: utf-8 -*-
"""
   Introduction to Deep Learning (LDA-T3114)
   Skeleton Code for Assignment 1: Sentiment Classification on a Feed-Forward Neural Network

   Hande Celikkanat & Miikka Silfverberg
"""

'\n   Introduction to Deep Learning (LDA-T3114)\n   Skeleton Code for Assignment 1: Sentiment Classification on a Feed-Forward Neural Network\n\n   Hande Celikkanat & Miikka Silfverberg\n'

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

from data_semeval import *
from paths import data_dir

In [7]:
torch.manual_seed(42)

In [176]:
#--- hyperparameters ---

N_CLASSES = len(LABEL_INDICES)
N_EPOCHS = 100
LEARNING_RATE = 0.05
BATCH_SIZE = 100
REPORT_EVERY = 1
IS_VERBOSE = True

In [192]:
def make_bow(tweet, indices):
    feature_ids = list(indices[tok] for tok in tweet['BODY'] if tok in indices)
    bow_vec = torch.zeros(len(indices))
    bow_vec[feature_ids] = 1
    return bow_vec.view(1, -1)

def generate_bow_representations(data):
    vocab = set(token for tweet in data['training'] for token in tweet['BODY'])
    vocab_size = len(vocab) 
    indices = {w:i for i, w in enumerate(vocab)}
  
    for split in ["training","development.input","development.gold",
                  "test.input","test.gold"]:
        for tweet in data[split]:
            tweet['BOW'] = make_bow(tweet,indices)

    return indices, vocab_size

# Convert string label to pytorch format.
def label_to_idx(label):
    return torch.LongTensor([LABEL_INDICES[label]])

In [199]:
class FFNN(nn.Module):
    def __init__(self, vocab_size, n_classes, n_hidden_1, n_hidden_2, n_hidden_3):
        super(FFNN, self).__init__()
        
        self.fc1 = nn.Linear(vocab_size, n_hidden_1)
        self.fc2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.fc3 = nn.Linear(n_hidden_2, n_hidden_3)
        self.fc4 = nn.Linear(n_hidden_3, n_classes)
        
    def forward(self, x):
        x = x.view(-1, vocab_size)
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))        
        out = F.relu(self.fc3(out))        
        out = F.relu(self.fc4(out))
        out = F.softmax(out, dim=1)
        return out

In [194]:
#--- data loading ---
data = read_semeval_datasets(data_dir)
indices, vocab_size = generate_bow_representations(data)

In [195]:
print(vocab_size)

22032


In [196]:
print(model)

FFNN(
  (fc1): Linear(in_features=22032, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=3, bias=True)
)


In [197]:
model = FFNN(vocab_size, N_CLASSES, n_hidden_1=500, n_hidden_2=200, n_hidden_3=100) #add extra arguments here if you use
loss_function = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [198]:
#--- training ---
for epoch in range(N_EPOCHS):
    total_loss = 0
    # Generally speaking, it's a good idea to shuffle your
    # datasets once every epoch.
    random.shuffle(data['training'])    

    for i in range(int(len(data['training'])/BATCH_SIZE)):
        minibatch = data['training'][i*BATCH_SIZE:(i+1)*BATCH_SIZE]
                
        x = [i['BOW'] for i in minibatch]
        labels = [label_to_idx(i['SENTIMENT']) for i in minibatch]
                
        x = torch.cat([i['BOW'] for i in minibatch], dim=-2)
        
        labels = torch.cat([label_to_idx(i['SENTIMENT']) for i in minibatch], dim=0)
                
        optimizer.zero_grad()
        predictions = model(x).view(100,3)
        
        loss = loss_function(predictions, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    if ((epoch+1) % REPORT_EVERY) == 0:
        print('epoch: %d, loss: %.4f' % (epoch+1, total_loss*BATCH_SIZE/len(data['training'])))


<ipython-input-193-613107b4a78d>:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(out)


epoch: 1, loss: 1.0918
epoch: 2, loss: 1.0798
epoch: 3, loss: 1.0708
epoch: 4, loss: 1.0636
epoch: 5, loss: 1.0582
epoch: 6, loss: 1.0546
epoch: 7, loss: 1.0520
epoch: 8, loss: 1.0502
epoch: 9, loss: 1.0492
epoch: 10, loss: 1.0482
epoch: 11, loss: 1.0477
epoch: 12, loss: 1.0471
epoch: 13, loss: 1.0473
epoch: 14, loss: 1.0470
epoch: 15, loss: 1.0467
epoch: 16, loss: 1.0465
epoch: 17, loss: 1.0466
epoch: 18, loss: 1.0461
epoch: 19, loss: 1.0461
epoch: 20, loss: 1.0459
epoch: 21, loss: 1.0460
epoch: 22, loss: 1.0456
epoch: 23, loss: 1.0451
epoch: 24, loss: 1.0449
epoch: 25, loss: 1.0445
epoch: 26, loss: 1.0443
epoch: 27, loss: 1.0434
epoch: 28, loss: 1.0427
epoch: 29, loss: 1.0421
epoch: 30, loss: 1.0410
epoch: 31, loss: 1.0397
epoch: 32, loss: 1.0385
epoch: 33, loss: 1.0365
epoch: 34, loss: 1.0343
epoch: 35, loss: 1.0316
epoch: 36, loss: 1.0286
epoch: 37, loss: 1.0245
epoch: 38, loss: 1.0198
epoch: 39, loss: 1.0149
epoch: 40, loss: 1.0100
epoch: 41, loss: 1.0040
epoch: 42, loss: 0.9939
e

In [200]:
#--- test ---
correct = 0
with torch.no_grad():
    for tweet in data['test.gold']:
        gold_class = label_to_idx(tweet['SENTIMENT'])
        gold_bow = tweet['BOW']
                
        prediction_tensor = model(gold_bow)
                
        predicted = prediction_tensor.max(1)[1]
        
        is_correct = predicted == gold_class
        
        if is_correct:
            correct += 1
        
        if IS_VERBOSE:
            print('TEST DATA: %s, GOLD LABEL: %s, GOLD CLASS %d, OUTPUT: %d' % 
                 (' '.join(tweet['BODY'][:-1]), tweet['SENTIMENT'], gold_class, predicted))

    print('test accuracy: %.2f' % (100.0 * correct / len(data['test.gold'])))

<ipython-input-193-613107b4a78d>:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(out)


tensor([[0.0393, 0.3215, 0.6392]])
TEST DATA: dec 21st 2012 will be know not as the end of the world but the baby boom ! #2012shit, GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 2
tensor([[0.0054, 0.9817, 0.0128]])
TEST DATA: mt @lccsy #syria , deir ezzor : ali bashar al-theeb was martyred . he was a soldier of the free syrian army . wednesday , 3 october 2012 ., GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 1
tensor([[0.0034, 0.0034, 0.9933]])
TEST DATA: @macmiller hate my life , because i can't see you at the roskilde festival on saturday , promise me to come back again , soon ., GOLD LABEL: negative, GOLD CLASS 0, OUTPUT: 2
tensor([[9.8911e-04, 9.8911e-04, 9.9802e-01]])
TEST DATA: excited for fri , not so i can get smashed but so i can go to the garden centre , look at the xmas display and have a coffee #wtf #grannyalert, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 2
tensor([[0.0232, 0.8221, 0.1546]])
TEST DATA: just got the ics update from sony on my sola ( apparently it was released on 

tensor([[0.0021, 0.0021, 0.9957]])
TEST DATA: @brookeofmidgard i'm sitting here with my belt , in jean shorts watching cena win his first title . i think we tie for 1st my friend xd, GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 2
tensor([[0.0011, 0.9978, 0.0011]])
TEST DATA: idk how im gone make it to total sports tomorrow for selection sunday, GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 1
tensor([[0.0200, 0.8054, 0.1746]])
TEST DATA: team ck vs demolition on november 17th at the american legion post 80 , main st . in binghamton ny . get your tickets now ! http://t.co/bieeefzd, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 1
tensor([[0.0310, 0.6078, 0.3612]])
TEST DATA: @charliewaters i laughed a little , but not sharing them out with anyone . will the weather be good tomorrow for boris bikes ?, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 1
tensor([[0.0010, 0.0010, 0.9980]])
TEST DATA: @gma my daughter @jordynking17 will be 14 on december 17th ! we r going to see buddy @ the kirby center in

tensor([[0.0322, 0.1246, 0.8431]])
TEST DATA: watching jiujitsu matches all day long , getting ready for sunday !, GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 2
tensor([[7.6873e-04, 9.9846e-01, 7.6873e-04]])
TEST DATA: the san diego #chargers expect wide receivers eddie royal and malcom #floyd to play in monday night's game against the denver #broncos, GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 1
tensor([[0.0244, 0.0672, 0.9084]])
TEST DATA: it only took 5 days for them to fix the broadband outage . china telecom may be improving ., GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 2
tensor([[0.0401, 0.5458, 0.4141]])
TEST DATA: omigod you guys i get to see holy motors tomorrow ., GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 1
tensor([[8.7137e-04, 9.9826e-01, 8.7137e-04]])
TEST DATA: everyday i'm like " shit is it the 17th yet ! ? did i miss the kiryu project ! ? " > <, GOLD LABEL: negative, GOLD CLASS 0, OUTPUT: 1
tensor([[0.0047, 0.9881, 0.0072]])
TEST DATA: becoming dangerously obsessed wit

tensor([[0.0038, 0.9909, 0.0052]])
TEST DATA: that should be andy lee's 3rd punt today down inside the 5 ., GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 1
tensor([[1.8859e-04, 9.9962e-01, 1.8859e-04]])
TEST DATA: rndm has a beef with music videos debut album " acts " out on oct 30 , 2012 https://t.co/ap2ynm2s, GOLD LABEL: negative, GOLD CLASS 0, OUTPUT: 1
tensor([[6.5344e-04, 6.5344e-04, 9.9869e-01]])
TEST DATA: @therobindiaz watching on livestream tonight you guys put on a hell of a show as always , see you in binghamton ny next saturday love u guys, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 2
tensor([[0.0055, 0.0055, 0.9890]])
TEST DATA: tomorrow is the mother's day ... i love you , mom ^ ___ ^, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 2
tensor([[0.0351, 0.2532, 0.7117]])
TEST DATA: @vivavhs haha . i've got to cut down mate , not looking forward to going through the warners . might try and sort small set tomorrow vestron ., GOLD LABEL: negative, GOLD CLASS 0, OUTPUT: 2
tensor([[9.

...

tensor([[0.0145, 0.0145, 0.9710]])
TEST DATA: miss teen usa on the next one .. saturday wahhh ? (:, GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 2
tensor([[0.0292, 0.0935, 0.8773]])
TEST DATA: i really want for there to be bball on sunday . and not just mysa, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 2
tensor([[4.5776e-05, 9.9991e-01, 4.5776e-05]])
TEST DATA: @ajaydevgn himmatwala set for holi release 29 march 2013 .. sure shot another blockbuster 4 ajay.it will going 2 3rd in row .. bb , sos , himmatwala, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 1
tensor([[2.3738e-04, 2.3738e-04, 9.9953e-01]])
TEST DATA: cold front in the metroplex ! happy friday ! :) http://t.co/tuvyo9kq, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 2
tensor([[0.0044, 0.9895, 0.0061]])
TEST DATA: @monzzz330 were fucking going to taco bell tomorrow, GOLD LABEL: negative, GOLD CLASS 0, OUTPUT: 1
tensor([[9.9382e-06, 9.9998e-01, 9.9382e-06]])
TEST DATA: i posted 23 photos on facebook in the album " abbey dawn at new y

tensor([[0.0022, 0.0022, 0.9957]])
TEST DATA: vanilla fudge , friday nite . tanner park copiague ny . free concert on the water . no mark stein . what an amazing ... http://t.co/ppsgr4cz, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 2
tensor([[0.0029, 0.0029, 0.9943]])
TEST DATA: last day of pssas tomorrow :) #yes #yeahbuddy ! ! !, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 2
tensor([[0.0016, 0.0016, 0.9969]])
TEST DATA: matta doesn't know how to view being ranked 4th in the country and 3rd in the big ten , except the conference must be really good ., GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 2
tensor([[0.0015, 0.0015, 0.9970]])
TEST DATA: @katvontease420 hmm i usually don't watch b / g shows but i may have to make an exception for something like that ;), GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 2
tensor([[0.0289, 0.7884, 0.1827]])
TEST DATA: new song by 50 cent performing " riot " remix with 2 chainz 2nd song off of 50 cent's upcoming gangsta grillz ... http://t.co/akbiphzj, GOLD

tensor([[6.9725e-04, 9.9861e-01, 6.9725e-04]])
TEST DATA: if we could just send taks scores instead of sat / act my life would be a billion-trillion times easier ., GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 1
tensor([[0.0384, 0.6286, 0.3330]])
TEST DATA: @babybaileybear [ he smiled ] it depends what type of books you like to read . i like james pattinson . they may not have those here .., GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 1
tensor([[4.0254e-06, 9.9999e-01, 4.0254e-06]])
TEST DATA: the band has just been announced as part of the line up for the rock am ring ( june 7th ) and rock im park ( june 8th ) festivals !, GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 1
tensor([[2.8957e-04, 9.9942e-01, 2.8957e-04]])
TEST DATA: from the soundboard will feature #georgecarlin and #paulmooney wed jul 11 3 to 5am summer fund drive . #donate #money @deepdancenyc, GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 1
tensor([[1.1480e-06, 1.0000e+00, 1.1480e-06]])
TEST DATA: " @kaiology : have u gotten ur

TEST DATA: remus lupin sat in the kitchen of 12 grimmauld place , eating a piece of pie molly had given him ., GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 1
tensor([[0.0334, 0.5233, 0.4433]])
TEST DATA: juniors playing powderpuff , if you aren't at practice tomorrow you will not play , it starts at 5:30 pm , hope to see you there !, GOLD LABEL: negative, GOLD CLASS 0, OUTPUT: 1
tensor([[0.0015, 0.9970, 0.0015]])
TEST DATA: i may have caught a ufo engaging warp drive in the background of this photo . http://t.co/asy4doye, GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 1
tensor([[0.0145, 0.9253, 0.0602]])
TEST DATA: @charlieismemate sweden eh ? how'd that come about ? not yet , sxsw will b n march . when is ur behind coming 2 the u . s . ? ;) how's the little 1 ?, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 1
tensor([[0.0013, 0.0013, 0.9974]])
TEST DATA: buongiorno a tutti ! sun is shining another beautiful day in vanuatu : ), GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 2
tensor([[8.4038e-04,

tensor([[0.0334, 0.3034, 0.6632]])
TEST DATA: news > flava - flava will be taking over floor 5 tomorrow night , with some of the best hip hop , afrobeats and rnb t http://t.co/pgquhnuy, GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 2
tensor([[0.0211, 0.0351, 0.9438]])
TEST DATA: night college course sorted , just have to enrole tomorrow now .. finally starting to figure out what i wanna do :-) #busyday, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 2
tensor([[0.0137, 0.9245, 0.0618]])
TEST DATA: for the 1st time in 30 years . for your splendiferous entertainment . arts in lechlade proudly presents an old ... http://t.co/qg0xaxtv, GOLD LABEL: positive, GOLD CLASS 2, OUTPUT: 1
tensor([[1.6843e-04, 9.9966e-01, 1.6843e-04]])
TEST DATA: @im_tgun @frayfenner @nvdox anyone else noticed the uncanny resemblance between tgun and petraeus ? tgun's dad may have a confession to make ., GOLD LABEL: neutral, GOLD CLASS 1, OUTPUT: 1
tensor([[0.0016, 0.0016, 0.9967]])
TEST DATA: nurses day - 12 may 2012 . n